# Random Forests


In [1]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (
    classification_report,
    r2_score,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
)
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from binning import bin

Read the train and test datasets


In [2]:
train_df = pd.read_csv("Training_set.csv")
test_df = pd.read_csv("Testing_set.csv")

Convert training set OHE columns to int8


In [3]:
columns_to_convert = train_df.columns[
    17:-1
]  # Columns from 17th column up to but excluding the last column
train_df[columns_to_convert] = train_df[columns_to_convert].astype("int8")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385516 entries, 0 to 385515
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: float64(12), int64(6), int8(4055)
memory usage: 1.5 GB


Convert testing set OHE columns to int8


In [4]:
columns_to_convert = test_df.columns[
    17:-1
]  # Columns from 17th column up to but excluding the last column
test_df[columns_to_convert] = test_df[columns_to_convert].astype("int8")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96379 entries, 0 to 96378
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: float64(12), int64(6), int8(4055)
memory usage: 385.9 MB


Get the training features and label


In [5]:
X_train = train_df.iloc[:, :-1].to_numpy()
y_train = train_df.iloc[:, -1].to_numpy()

Get the testing features and label


In [6]:
X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

Initialize a 5-fold cross-validation object


In [7]:
kf = KFold(n_splits=5)

## Random Forests Classifier


Bin the label


In [8]:
y_train_binned = bin(y_train)
y_test_binned = bin(y_test)

Initialize a RandomForestClassifier


In [27]:
rfc = RandomForestClassifier(n_jobs=4, max_depth=50)

Initialize scores lists


In [28]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

Train the model using 5-fold cross-validation


In [29]:
fold = 1
for train, test in kf.split(X_train, y_train_binned):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    rfc.fit(X_train[train], y_train_binned[train])

    # Predict on the test set
    predictions = rfc.predict(X_train[test])

    # Evaluate the model
    precision = precision_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train_binned[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
Precision = 0.5109707808549148
Recall = 0.47000155633948953
Accuracy = 0.47000155633948953
F1 score = 0.3819352272676676

##### FOLD: 2 #####
Precision = 0.4740931428016712
Recall = 0.4692554115922856
Accuracy = 0.4692554115922856
F1 score = 0.37951330530442595

##### FOLD: 3 #####
Precision = 0.4636418285280089
Recall = 0.4688663216736054
Accuracy = 0.4688663216736054
F1 score = 0.3786154789963096

##### FOLD: 4 #####
Precision = 0.4873783986947767
Recall = 0.47153807244854284
Accuracy = 0.47153807244854284
F1 score = 0.3819436491946871

##### FOLD: 5 #####
Precision = 0.4908168089420688
Recall = 0.47044862067623827
Accuracy = 0.47044862067623827
F1 score = 0.38140625802482275



Print the mean scores of the folds


In [30]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.485380191964288
Mean Recall = 0.4700219965460323
Mean Accuracy = 0.4700219965460323
Mean F1 score = 0.3806827837575826


Make predictions on the test set


In [33]:
predictions = rfc.predict(X_test)

Evaluate the model


In [34]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.24      0.00      0.00     11718
           1       0.47      0.88      0.61     42216
           2       0.46      0.25      0.32     32447
           3       0.73      0.01      0.01      9998

    accuracy                           0.47     96379
   macro avg       0.47      0.28      0.24     96379
weighted avg       0.47      0.47      0.38     96379



## Random Forests Regressor


Initialize a RandomForestRegressor


In [13]:
rfr = RandomForestRegressor(n_jobs=4)

Initialize scores lists


In [14]:
r2_scores = []

Train the model using 5-fold cross-validation


In [15]:
fold = 1
for train, test in kf.split(X_train, y_train):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    rfr.fit(X_train[train], y_train[train])

    # Predict on the test set
    predictions = rfr.predict(X_train[test])

    # Evaluate the model
    r2 = r2_score(y_true=y_train[test], y_pred=predictions)

    # Store the result
    r2_scores.append(r2)

    # Print the scores for each fold
    print(f"R2 score = {r2}")

    fold += 1

##### FOLD: 1 #####
R2 score = 0.10097462198691853
##### FOLD: 2 #####
R2 score = 0.08919078099226141
##### FOLD: 3 #####
R2 score = 0.09476152873124877
##### FOLD: 4 #####
R2 score = 0.09542324038510774
##### FOLD: 5 #####
R2 score = 0.09483583931662942


Print the mean scores of the folds


In [16]:
print(f"Mean R2 score = {np.mean(r2_scores)}")

Mean R2 score = 0.09503720228243318


Make predictions on the test set


In [17]:
predictions = rfr.predict(X_test)

Evaluate the model


In [18]:
print(r2_score(y_true=y_test, y_pred=predictions))

0.08964064245552128
